#  TrafficGeneration
<i>Adapted for use with FABRIC from [Traffic Generation](https://www.cs.unc.edu/Research/geni/geniEdu/08-TrafficGen.html)</i>
        
<b> Prerequisites  </b>
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-5 at https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md#section-1-get-started.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md)

In this experiment you will be introduced to the principles of traffic generation using Nping. This is the second step in this assignment. To go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb)

## 1. Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.
#### Import the Fabric API

In [ ]:
# Load Fablib and Node Information
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                    
fablib.show_config()
import json
import traceback

slice_name = "TrafficGeneration"
try:
    slice = fablib.get_slice(slice_name)
    print(f"Slice: {slice.get_name()}, {slice.get_state()}")
    slice.list_nodes()
except Exception as e:
    print(f"Get Slices Fail: {e}")

## 2 Guided Experiment
<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/TrafficGeneration/figures/TrafficGenTop.png?raw=true"><br>
### 2.1 Running Nping
1. Open an ssh terminal to each of the node

Nping comes from the Nmap package and is self-contained. We will use this to generate traffic across the nodes. nping has different methods of running traffic. We will use tcp traffic using "--tcp" or udp with "--udp"

2. We can start by running a simple test on the network with the following command ```sudo nping --tcp -p 5000 10.0.0.2```

However we can also specify how many rounds we want to use with the "-c" option, can specify the length of the package "--data-length" and delay between sent packets "delay"

3. For this experiment we will run traffic generation using TCP traffic on the port 5000, for 4000 rounds, and we will start with data length of 1 and no delay. The command we will use is
   <br> ```sudo nping --tcp --dest-ip 10.0.0.2 -p 5000 --privileged -c 40000 --data-length 1024 --delay 0 ```
4. For more information about Nping: https://nmap.org/nping/
        


## 3. Assignment
### 1 Running Nping
1. Design an experiment in which you would use Nping for Traffic Generation.
2. What would your experiment test?
3. Why would realistic traffic be desired?
4. What traffic source would you collect a header trace from?


### 2 Running Nping
1. Open Two terminals to node 1. On one of the terminals start the script using the following command ```./TG-output.sh```
2. On the second terminal generate similar traffic using a command similar to the ones during the experiment
3. Once the traffic generated on the second terminal is completed stop catching packets by running ```Ctrl-C``` on the first terminal (Keep in mind that depending on the length and time of the traffic you generated, this step might take some time)
4. Use the following code block to download a graph of the throughput that Nping creates (with your command) as it runs.

In [ ]:
# Download chart from Node 1
try:
    slice = fablib.get_slice(slice_name)
    A = slice.get_node('Node_1')
    A.execute("Rscript TG-data-analysis.R")
    A.download_file("Traffic-TG.svg","Traffic-TG.svg")
except Exception as e:
    print(f"Fail: {e}")

## Cleanup Resources
Once you have completed the assignment shut down the slice

In [ ]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True". This is to prevent accidental slice deletion
    CHECK = False
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")